In [47]:
##Inspired by https://towardsdatascience.com/automate-excel-reporting-with-python-233dd61fb0f2
import pandas as pd

from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.chart import BarChart, Reference
from openpyxl.chart.layout import Layout, ManualLayout

In [48]:
df = pd.read_excel('TestSalesdata.xlsx', parse_dates=['Order Date'])
print(df.head())

  Order Date    Sales            Product Sub-Category Customer Segment  \
0 2009-01-18  2842.54  Binders and Binder Accessories        Corporate   
1 2009-01-27    55.89                    Rubber Bands        Corporate   
2 2009-02-04  1441.57            Computer Peripherals        Corporate   
3 2009-02-22  1633.37             Pens & Art Supplies        Corporate   
4 2009-04-20   225.94            Computer Peripherals        Corporate   

  Product Category  
0  Office Supplies  
1  Office Supplies  
2       Technology  
3  Office Supplies  
4       Technology  


In [28]:
print(df.dtypes)

Order Date              datetime64[ns]
Sales                          float64
Product Sub-Category            object
Customer Segment                object
Product Category                object
dtype: object


In [55]:
filtered = df[df['Product Category'] == 'Technology']
quarterly_sales = pd.pivot_table(filtered, index = filtered['Order Date'].dt.quarter, columns = 'Product Sub-Category', values = 'Sales', aggfunc='sum')

print("Quarterly Sales Pivot Table:")
print(quarterly_sales.head())

Quarterly Sales Pivot Table:
Product Sub-Category  Computer Peripherals  Copiers and Fax  Office Machines  \
Order Date                                                                     
1                                 52123.32         32003.59        393451.34   
2                                 76734.12          9138.39         63146.36   
3                                 34772.86         68792.00        153569.91   
4                                 51934.77        163352.88        122337.84   

Product Sub-Category  Telephones and Communication  
Order Date                                          
1                                       86287.9370  
2                                       95025.7160  
3                                      175163.8095  
4                                      123990.6390  


In [56]:
quarterly_sales.to_excel( "QuarterlySales.xlsx", engine='xlsxwriter', startrow=3, sheet_name="Quarterly Sales")

In [57]:
wb = load_workbook("QuarterlySales.xlsx")
sheet1 = wb['Quarterly Sales']

#Format the First Sheet
sheet1['A1'] = 'Quarterly Sales'
sheet1['A4'] = 'Quarter'

sheet1['A1'].style = 'Title'
sheet1['A2'].style = 'Headline 2'

for i in range(5, 9):
    sheet1[f'B{i}'].style='Currency'
    sheet1[f'C{i}'].style='Currency'
    sheet1[f'D{i}'].style='Currency'
    sheet1[f'E{i}'].style='Currency'

#Add the Bar Chart
bar_chart = BarChart()
data = Reference(sheet1, min_col=2, max_col=5, min_row=4, max_row=8)
categories = Reference(sheet1, min_col=1, max_col=1, min_row=5, max_row=8)
bar_chart.add_data(data, titles_from_data=True)
bar_chart.set_categories(categories)
bar_chart.style = 12
bar_chart.y_axis.title = 'Product Category'
bar_chart.x_axis.title = 'Quarter'
bar_chart.legend.position = 'b'
sheet1.add_chart(bar_chart, "F4")

bar_chart.title = 'Sales by Product Category'


from copy import deepcopy

hbar_chart = deepcopy(bar_chart)
hbar_chart.style = 13
hbar_chart.type = "bar"
hbar_chart.title = "Horizontal Bar Chart"
hbar_chart.layout = Layout(
    ManualLayout(
    x=0.25, y=0.25,
    h=0.5, w=0.5,
    xMode="edge",
    yMode="edge",
    )
)

sheet1.add_chart(hbar_chart, "O4")

wb.save("QuarterlySales.xlsx")